In [ ]:
%pip install huggingface_hub --quiet

In [ ]:
%pip install jsonlines --quiet

In [ ]:
%pip install -U datasets --quiet

In [ ]:
%pip install -U peft --quiet

In [ ]:
%pip install -U transformers --quiet

In [ ]:
%pip install -U rouge_score --quiet

In [ ]:
%pip install evaluate==0.4.3 --quiet

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
import torch
model_ID = 'google/paligemma-3b-mix-448'
dtype = torch.bfloat16
processor = AutoProcessor.from_pretrained(model_ID)
model = PaliGemmaForConditionalGeneration.from_pretrained(model_ID)

In [ ]:
from datasets import load_dataset

dataset = load_dataset("amohseni/receipt_VLM_information_extraction")
dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
predictions = []
targets = dataset['test'][0:]['suffix']
instructions = dataset['test'][0:]['prefix']
for i in range(len(targets)):
    prompt = "<image>" + instructions[i]
    image_file = dataset['test'][i]['image'].convert("RGB")
    model_inputs = processor(text=prompt, images=image_file, return_tensors="pt").to(dtype).to(device)
    input_len = model_inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**model_inputs, max_new_tokens=512, do_sample=False)
        generation = generation[0][input_len:]
        decoded = processor.decode(generation, skip_special_tokens=True)
        predictions.append(decoded)
        print(i)

In [ ]:
import pandas as pd
zipped_performance = list(zip(targets, predictions))
df = pd.DataFrame(zipped_performance, columns = ['human_baseline', 'model extraction'])
df.head()

In [ ]:
df.to_csv('original_model_extraction_performance.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv('./original_model_extraction_performance.csv')
human_base_line_extraction = df['human_baseline']
original_model_extraction = df['model extraction']

In [ ]:
import evaluate
import numpy as np
rouge = evaluate.load('rouge')
original_model_results = rouge.compute(
    predictions=original_model_extraction,
    references=human_base_line_extraction[0:len(original_model_extraction)],
    use_aggregator=True,
    use_stemmer=True,
)
print('ORIGINAL MODEL:')
print(original_model_results)

In [ ]:
performance = pd.DataFrame.from_dict([original_model_results])
performance.head()
performance.to_csv('./Rouge_original_model_extraction.csv', index=False)